# **PREAnoTeTool - ETAPA 1: PRÉ-ANOTAÇÃO DE TEXTOS**

## Caderno resposável por pré-anotar os textos do corpus e interoperar com a ferramenta Doccano. De acordo com o arquivo doutrinário de entrada, utilizando o metamodelo IDEA-C2, é possível pré-anotar entidades e relações alinhada à abordagem supervisionada à distância por meio de expressões regulares. Como resultado, o arquivo JSON gerado contém os termos anotados para ser importado pela ferramenta Doccano.

### Passo a passo
#### 1 - "Setar" o caminho /content/drive/MyDrive/preanotetool/
##### 1.1 - Arquivos de Entrada (Glossário): .../docs/C-20-1- Glossário do EB.pdf
##### 1.2 - Arquivo de Saída
###### 1.2.1 - JSONL: .../texts/output_ajustado.jsonl
###### 1.2.2 - TXT: .../texts/entidades.txt
###### 1.2.3 - TXT: .../texts/relations.txt
##### 1.4 - Observação: O arquivo de saída JSONL deve ser importado no Doccano.

### Data de Criação: 01-10-2023
### Última atualização: 20-11-2023
### Autores: Gustavo Danon e Jones.

In [1]:
# Monta o drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install PyPDF2

In [6]:
# Setando a pasta onde estão localizados os outros arquivos .py que auxiliam a execução
import sys
sys.path.insert(0,'/content/drive/MyDrive/preanotetool/')
import os
# # tem que setar o diretório
os.chdir('/content/drive/MyDrive/preanotetool/')
cwd = os.getcwd()
#print(cwd)

'''
O notebook está especificado para cada regex, então mudar na mao os regex de:
    page_regex : as quebras de página do arquivo (indice da pagina ou ordenacao do glossario)
    term_regex : o regex para extrair os sujeitos
    delimiter_regex : o regex para substituir pelo predicado
Mudar tambem page_start e page_end para pular o glossario e o fim do arquivo
'''

import pdf_functions
import terms_functions
from find_all import *
import enrich_relation as ER

import re
import sys
import json

import math

In [ ]:
# Função para retirar caracteres indesejáveis
def limpar_texto(text):
  # text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
  # text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  # text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
  # text = re.sub(r" +", ' ', text)
  #text = re.sub(".", "", text)
  text = re.sub("/", " ", text)
  text = re.sub("[()]/", " ", text)
  #text = text.lower()
  return text

In [ ]:
# File info
pdf_path = 'docs/C-20-1- Glossário do EB.pdf'

pdf_text = pdf_functions.pdf_to_text(pdf_path, page_start=12, page_end=363)
raw_lines = pdf_text.split('\n')

for i in range(len(raw_lines)):
  #raw_lines[i] = limpar_texto(raw_lines[i])
  raw_lines[i] = raw_lines[i]

print(raw_lines)

# Change page regex here to format them correctly
#**********************************************************************************
page_regex = r'\A[A-ZÀ-Ú][-–](.*)$|\A[A-ZÀ-Ú]$|\A[A-ZÀ-Ú] ([0-9]*)-([0-9]*)$'
page_regex = re.compile(page_regex)

lines = pdf_functions.filter_page_number(raw_lines, page_regex)

delimiter_regex = r'[-–˗⁻−﹣](\s*)((1\.\s)?)[A-ZÀ-Ú][a-zà-ú ]'
term_regex = r'^[A-ZÀ-ÖØ-Þ\(\)\s\/,-]{2,}'
#**********************************************************************************

['C 20-1', 'A', 'ABALROAMENTO – Choque entre duas ou mais - ou de uma com duas ou', 'mais - aeronaves, em vôo ou em manobra na superfície, ou embarcações de', 'qualquer espécie, em porto, local de atracação, ou via navegável, e do qual', 'resultem danos.', 'ABASTECIMENTO – Conjunto de atividades que tem o propósito de prever', 'e prover, para as forças e organizações militares, o material necessário a mantê-', 'las em condições de plena eficiência. Assim, o abastecimento proporciona um', 'fluxo adequado do material necessário, desde as fontes de obtenção até as', 'organizações militares consumidoras, abrangendo a função logística suprimento', 'e parte da função logística transporte, além de relacionar-se, estreitamente, com', 'a função logística manutenção. Ver FUNÇÃO LOGÍSTICA SUPRIMENTO.', 'ABICAR – Manobra realizada por navios ou embarcações, encalhando', 'intencionalmente com a proa em uma praia ou margem, de maneira controlada,', 'com o fim de desembarcar pessoal ou material.', 'A

## Estratégia de Geração do JSONL - 1
### Código que implementa a geração do arquivo .JSONL, fixando um termo por documento
### Depois que rodar esse script, tem que rodar o outro script de ajuste dos IDs

In [ ]:
# Variáveis
# FIRST_TERM = 0   (Tem que ser inicializada com 0)
# TERMOS_DIVISAO = 1 (Esta variável define a quantidade de termos que será paginado no Doccano. Se quiser que sejam 5 termos por página, a variável deve ser inicializada com valor 5)
# TOTAL_TERMOS = Y (Deve-se verificar a quantidade de termos a serem processados. No Glossário, por exemplo, são 3294. Logo, a variável é inicializada com este valor.)

# ****************************************************************
# Inicialização de variáveis
# ****************************************************************
JSONL_COMPLETO=[]
FIRST_TERM = 0
TERMOS_DIVISAO = 1  #  Variável que indica a quantidade de termos o usuário quer fatiar por documento
LAST_TERM = TERMOS_DIVISAO
TOTAL_TERMOS = 3294
TOTAL_DIVISAO=math.floor(TOTAL_TERMOS/TERMOS_DIVISAO)
i=0
# ****************************************************************

while i<TOTAL_DIVISAO:
    terms, all_formated_lines = terms_functions.terms_and_lines(lines, term_regex, delimiter_regex)
    terms = list(terms.values())

    formated_lines = all_formated_lines[FIRST_TERM:LAST_TERM]

    relation_terms = find_relation_terms(**{
        'lines': formated_lines,
        'terms': terms,
        'term_index': max(tf.get_ids(terms)) + 1,
        'slice_term_index': FIRST_TERM
    })

    main_relations = find_main_relations(**{
      'lines': formated_lines,
      'terms': terms,
      'relation_terms': relation_terms,
      'relation_type' : 'associated_with', #tive que inserir isso aqui pra ele aceitar o parâmetro com o novo valor
      'slice_term_index': FIRST_TERM
    })

# São definidas as expressões regulares que devem ser utilizadas no glossário para buscadas entre os pares de entidades
# Exemplo:
# Texto = "COMANDANTE DE AERONAVE - Membro da tripulação responsável pela aeronave
# Pares de entidade:  COMANDANTE DE AERONAVE, aeronave
# Relação: "responsible_for"
# Expressão regular associada ao "responsible_for" = "responsavel p"

    relation_models = [
            ER.Relations('. Ver ', 'associated_with'),
            ER.Relations('. O mesmo que ', 'equivalent_to'),
            ER.Relations(' tipo de ', 'type_of'),
            ER.Relations(' subunidade ', 'type_of'),
            ER.Relations(' são as seguintes ', 'type_of'),
            ER.Relations(' é a seguinte ', 'type_of'),
            ER.Relations(' são el ', 'type_of'),
            ER.Relations(' executado p ', 'responsible_for'),
            ER.Relations(' responsável ', 'responsible_for'),
            ER.Relations(' emprega', 'responsible_for'),
            ER.Relations(' avalia ', 'responsible_for'),
            ER.Relations(' designad ', 'responsible_for'),
            ER.Relations(' dirig ', 'responsible_for'),
            ER.Relations(' coordena ', 'responsible_for'),
            ER.Relations(' confronto dialético', 'responsible_for'),
            ER.Relations(' estabelecid', 'responsible_for'),
            ER.Relations(' composto p ', 'composed_of'),
            ER.Relations(' conjunto d ', 'composed_of'),
            ER.Relations(' capacidade d ', 'capacity_of'),
            ER.Relations(' capaz d', 'capacity_of'),
            ER.Relations(' apoiar', 'capacity_of'),
            ER.Relations(' atribuíd', 'capacity_of'),
            ER.Relations(' orienta ', 'capacity_of'),
            ER.Relations(' ocorr ', 'occurs_in'),
            ER.Relations(' localiza ', 'occurs_in'),
            ER.Relations(' emprega ', 'applied_to'),
            ER.Relations(' aplica ', 'applied_to'),
            ER.Relations(' usa ', 'applied_to'),
            ER.Relations(' utiliza ', 'applied_to')

    ]

    jsonString = {
          "id": i,
          "text": ' '.join(formated_lines)
    }

    ER.enrich_relations('\n'.join(all_formated_lines), relation_models, main_relations)

    # Ajustar o offset para fatiar p texto
    first_offset = terms[FIRST_TERM]['start_offset']
    terms_functions.shift_terms(terms, first_offset)
    terms_functions.shift_terms(relation_terms, first_offset)

    jsonRelations = {"relations": []}
    all_terms = list()

    jsonRelations['relations'].extend(main_relations)

    jsonEntities = {
        "entities": list()
    }

    jsonEntities['entities'].extend(terms[FIRST_TERM:LAST_TERM])
    jsonEntities['entities'].extend(relation_terms)

    jsonString.update(jsonEntities)
    jsonString.update(jsonRelations)
    jsonString.update({"Comments": []})

    # Escrita dos JSON
    jsonFile = ''
    jsonFile = 'texts/output.jsonl'

    if jsonString != '':
      JSONL_COMPLETO.append(jsonString)

    FIRST_TERM = LAST_TERM
    LAST_TERM = LAST_TERM + 1

    if LAST_TERM > TOTAL_TERMOS:
      LAST_TERM = TOTAL_TERMOS
      j=TOTAL_DIVISAO
    i=i+1
#  Fim do While
if JSONL_COMPLETO != '':
  jsonFile = 'texts/output.jsonl'
  with open(jsonFile, 'w', encoding='utf-8') as json_file:
    for ddict in JSONL_COMPLETO:
        jout = json.dumps(ddict) + '\n'
        json_file.write(jout)
  print(f"Resultados salvos em {jsonFile}")



Resultados salvos em texts/output.jsonl


## Script complementar e fundamental para a geração do JSONL responsável por ajustar os IDs - São extraídas as entidades (entidades.txt) e as relações (relations.txt)

In [ ]:
nome_arquivo_default='output'
data_file_path = '/content/drive/MyDrive/preanotetool/texts/'+ nome_arquivo_default+ '.jsonl'
DATA_FILE_PATH_JSON_OUT = '/content/drive/MyDrive/preanotetool/texts/'+ nome_arquivo_default+ '_ajustado.jsonl'
NOME_ARQUIVO_OUTPUT_ENTIDADES = "/content/drive/MyDrive/preanotetool/texts/entidades.txt"
NOME_ARQUIVO_OUTPUT_RELATIONS = "/content/drive/MyDrive/preanotetool/texts/relations.txt"

cont_doc = 0
cont_ent = 0
cont_rel = 0
json_list = []
map_ent = {}
ent_n3=[]
rels_n3=[]
rels_n4=[]
map_ent1 = {}
map_ent2 = {}

with open(NOME_ARQUIVO_OUTPUT_ENTIDADES, 'w') as output:
  output.write('\n'.join(ent_n3))

with open(data_file_path, 'r', encoding='utf-8') as jsonl_file:
    json_total_list = []
    num_lines = 0

    for line in jsonl_file:
        json_total_list.append(json.loads(line))
        num_lines += 1

#print(json_total_list)

#
for cont_doc, docum in enumerate(json_total_list):
    lista_entidades = []
    lista_relations = []
    text = docum['text']
    entities =  docum['entities']
    relations =  docum['relations']
    for ent in entities:
        #print(ent)
        ent_id = ent['id']
        ent_start = ent['start_offset']
        ent_end = ent['end_offset']
        ent_label = ent['label']

        #ent_texto = ent['texto_key']
        ent_texto = text[ent_start:ent_end]

        dict_entidades = {
            "id": cont_ent,
            "label": ent_label,
            "start_offset": ent_start,
            "end_offset": ent_end,
            "texto_key": ent_texto
        }

        # ENTIDADES
        ent_n3.append(ent_texto.lower())
        lista_entidades.append(dict_entidades)
        map_ent[ent_id] = cont_ent
        map_ent1[ent_id] = ent_texto
        cont_ent += 1

    for rel in relations:  # "id": 1740, "from_id": 82887, "to_id": 82885, "type": "usado_para"
        #print(rel)
        rel_id = rel['id']
        rel_from_id = map_ent[rel['from_id']]
        rel_to_id = map_ent[rel['to_id']]
        rel_type = rel['type']

        rel_from_id_texto= map_ent1[rel['from_id']]
        rel_to_id_texto = map_ent1[rel['to_id']]

        #print('(', rel_from_id, '-', rel_to_id, ')', '-', rel_type)

        dict_relations = {
            "id": cont_rel,
            "from_id": rel_from_id,
            "to_id": rel_to_id,
            "type": rel_type
        }

        #rels_n3.append([cont_rel, rel_from_id_texto, rel_type, rel_to_id_texto])
        #print(rel_from_id_texto, '-', rel_type, '-', rel_to_id_texto, '\n')
        rels_n3.append([cont_rel, rel_from_id_texto, rel_type, rel_to_id_texto])
        lista_relations.append(dict_relations)
        cont_rel += 1

    dict_anotacoes = {
        "id": cont_doc,
        "text": text,
        "entities": lista_entidades,
        "relations": lista_relations,
        "Comments": []
    }

    json_list.append(dict_anotacoes)

    ent_n3_final = list(set(ent_n3))

with open(DATA_FILE_PATH_JSON_OUT, 'w') as f_saida:
    for ddict in json_list:
        jout = json.dumps(ddict,ensure_ascii=False) + '\n'
        f_saida.write(jout)

with open(NOME_ARQUIVO_OUTPUT_ENTIDADES, 'w') as output:
  output.write('\n'.join(ent_n3_final))

rels_n4.append(rels_n3)
with open(NOME_ARQUIVO_OUTPUT_RELATIONS, 'w') as output:
  output.write(','.join(map(str, rels_n4)))